# 기계독해 모델 학습

> 작성자      
```
* 김성현 (bananaband657@gmail.com)  
김바다 (qkek983@gmail.com)
박상희 (parksanghee0103@gmail.com)  
이정우 (jungwoo.l2.rs@gmail.com)
```
[CC BY-NC-ND](https://creativecommons.org/licenses/by-nc-nd/2.0/kr/)

In [1]:
!pip install transformers

     |████████████████████████████████| 2.1MB 8.7MB/s 
     |████████████████████████████████| 3.3MB 49.3MB/s 
     |████████████████████████████████| 901kB 49.6MB/s 


In [2]:
import torch
import sys

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [4]:
!mkdir dataset

In [5]:
!wget https://korquad.github.io/dataset/KorQuAD_v1.0_train.json
!wget https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json


--2021-04-21 00:49:31--  https://korquad.github.io/dataset/KorQuAD_v1.0_train.json
Resolving korquad.github.io (korquad.github.io)... 185.199.108.153, 185.199.111.153, 185.199.110.153, ...
Connecting to korquad.github.io (korquad.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38527475 (37M) [application/json]
Saving to: ‘KorQuAD_v1.0_train.json’

KorQuAD_v1.0_train. 100%[===================>]  36.74M   111MB/s    in 0.3s    

2021-04-21 00:49:32 (111 MB/s) - ‘KorQuAD_v1.0_train.json’ saved [38527475/38527475]

--2021-04-21 00:49:32--  https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json
Resolving korquad.github.io (korquad.github.io)... 185.199.108.153, 185.199.111.153, 185.199.110.153, ...
Connecting to korquad.github.io (korquad.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3881058 (3.7M) [application/json]
Saving to: ‘KorQuAD_v1.0_dev.json’

KorQuAD_v1.0_dev.js 100%[======

In [6]:
!mv /content/KorQuAD_v1.0_train.json dataset
!mv /content/KorQuAD_v1.0_dev.json dataset

In [7]:
# 데이터셋을 파싱해오겠습니다 :-)

import json
from pathlib import Path

def read_squad(path):
    path = Path(path)
    with open(path, 'rb') as f:
        squad_dict = json.load(f)

    contexts = []
    questions = []
    answers = []
    for group in squad_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                for answer in qa['answers']:
                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer)

    return contexts, questions, answers


train_contexts, train_questions, train_answers = read_squad('dataset/KorQuAD_v1.0_train.json')
val_contexts, val_questions, val_answers = read_squad('dataset/KorQuAD_v1.0_dev.json')

In [8]:
print(train_contexts[52471])
print(len(train_contexts[52471]))

사우스런던 브릭스톤 태생 보위는 어릴 적 음악에 대한 관심을 키워왔으며, 결국에는 예술, 음악, 디자인을 배워 전문적인 음악가 경력을 1963년부터 착수했다. 〈Space Oddity〉는 1969년 7월 발표 뒤 영국 음반 차트에서 상위 5위에 오른 그의 첫 싱글이다. 실험 활동을 거친 그는 1972년 음악을 재개, 자신의 글램록 시기 동안 이색적이고 양성적인 제2의 자아인 지기 스타더스트로 활동을 이어나갔다. 성공을 거둔 싱글 〈Starman〉과 전 세계적인 인기를 끈 음반 《The Rise and Fall of Ziggy Stardust and the Spiders from Mars》으로 캐릭터를 내세운 보위는 1975년 "플라스틱 솔"을 캐릭터화시켜 자신을 철저히 바꾼다. 이 행동은 당초 영국에서 그의 열혈팬의 반발을 샀으나 미국에서는 싱글 〈Fame〉과 음반 《Young Americans》을 통해 처음으로 메이저한 성공을 거두게 된다. 1976년 보위는 컬트 영화 《지구에 떨어진 사나이》에 출연하고 음반 《Station to Station》을 발표한다. 이듬해에는 일렉트릭 음악을 접목한 음반 《Low》 (1977)을 발표하면서 음악적 예상을 깨뜨렸다. 이 음반은 브라이언 이노와의 세 번의 협업 중 첫 번째로 이는 이후 "베를린 삼부작"으로 일컬어진다. 뒤를 이어 발표된 《"Heroes"》 (1977)와 《Lodger》 (1979)는 영국 차트 상위 5위에 진입, 지속적인 극찬을 받았다.
734


In [9]:
print(train_questions[52471])

보위가 1977년 일렉트릭 음악을 접목하여 발표한 음반은?


In [10]:
print(train_answers[52471]) # 본문 내에서 정답이 시작되는 음절 순서가 'answer start'에 저장되어 있습니다.

{'text': 'Low', 'answer_start': 568}


In [11]:
def add_end_idx(answers, contexts):
    for answer, context in zip(answers, contexts):
        # 모델 학습을 위해 정답 데이터를 만들겠습니다.
        # 정답 데이터는 start음절과 end 음절로 구성되어 있습니다.
        # 모델은 전체 토큰 중에서 start token과 end token을 찾아내는 것을 목표로 학습하게 됩니다.
        gold_text = answer['text']
        start_idx = answer['answer_start']
        end_idx = start_idx + len(gold_text)
        

        # sometimes squad answers are off by a character or two – fix this
        # 실제 본문에서 해당 음절 번호로 잘라냈을 때, 정답과 같은지 검사해서 start, end를 보정합니다 :-)
        # '이순신은 조선 중기의 무신이다' -> '이순신' -> start: 0, end: 4
        if context[start_idx:end_idx] == gold_text:
            answer['answer_end'] = end_idx
        elif context[start_idx-1:end_idx-1] == gold_text:
            answer['answer_start'] = start_idx - 1
            answer['answer_end'] = end_idx - 1     # When the gold label is off by one character
        elif context[start_idx-2:end_idx-2] == gold_text:
            answer['answer_start'] = start_idx - 2
            answer['answer_end'] = end_idx - 2     # When the gold label is off by two characters
    return answers

train_answers = add_end_idx(train_answers, train_contexts)
val_answers = add_end_idx(val_answers, val_contexts)

In [12]:
from transformers import AutoModel, AutoTokenizer, BertTokenizer
MODEL_NAME = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True)
val_encodings = tokenizer(val_contexts, val_questions, truncation=True, padding=True)

In [13]:
def add_token_positions(encodings, answers):
    start_positions = []
    end_positions = []
    # 이제 음절 index를 token index와 mapping하는 작업을 해보도록 하겠습니다 :-)
    for i in range(len(answers)):
        # tokenizer의 char_to_token 함수를 호출하면 음절 숫자를 token index로 바꿔줄 수 있습니다.
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end']))
        # 아래 부분은 truncation을 위한 과정입니다.
        # if start position is None, the answer passage has been truncated
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length

        # if end position is None, the 'char_to_token' function points to the space before the correct token - > add + 1
        if end_positions[-1] is None:
            end_positions[-1] = encodings.char_to_token(i, answers[i]['answer_end'] + 1)

        # 추가된 예외 처리, 예를들어서 tokenizer와 model input의 max_length가 512인데, start와 end position이 600과 610 이면 둘다 max_length로 변경해야함.
        # 어차피 max_length가 512인 모델은 정답을 볼 수 없음.
        if start_positions[-1] is None or start_positions[-1] > tokenizer.model_max_length:
            start_positions[-1] = tokenizer.model_max_length
        
        if end_positions[-1] is None or end_positions[-1] > tokenizer.model_max_length:
            end_positions[-1] = tokenizer.model_max_length

    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})
    return encodings

train_encodings = add_token_positions(train_encodings, train_answers)
val_encodings = add_token_positions(val_encodings, val_answers)

In [14]:
import torch

class SquadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

train_dataset = SquadDataset(train_encodings)
val_dataset = SquadDataset(val_encodings)

In [15]:
from transformers import BertForQuestionAnswering
model = BertForQuestionAnswering.from_pretrained(MODEL_NAME)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-bas

In [16]:
from transformers import BertForQuestionAnswering, Trainer, TrainingArguments
import sys
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=1,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    logging_dir='./logs',            # directory for storing logs
    logging_steps=100,
    learning_rate=3e-5,
    save_total_limit=5
)

In [17]:
model = BertForQuestionAnswering.from_pretrained(MODEL_NAME)
model.to(device)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset            # evaluation dataset
)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-bas

In [18]:
trainer.train() # 1 epoch에 대략 1시간 정도 걸립니다.

Step,Training Loss
100,2.902900
200,1.333700
300,1.222500
400,1.083600
500,1.059100
600,0.996000
700,0.978300
800,0.933300
900,1.016500
1000,0.879300


TrainOutput(global_step=7551, training_loss=0.7338287806671955, metrics={'train_runtime': 7377.5772, 'train_samples_per_second': 1.024, 'total_flos': 3.2895005998393344e+16, 'epoch': 1.0, 'init_mem_cpu_alloc_delta': 0, 'init_mem_gpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': -1030262784, 'train_mem_gpu_alloc_delta': 2197022720, 'train_mem_cpu_peaked_delta': 1042755584, 'train_mem_gpu_peaked_delta': 6530332672})

In [19]:
trainer.save_model(".")

In [20]:
from transformers import pipeline

In [21]:
nlp = pipeline("question-answering", model=model, tokenizer=tokenizer, device=0)

context = r"""
이순신(李舜臣, 1545년 4월 28일 ~ 1598년 12월 16일 (음력 11월 19일))은 조선 중기의 무신이었다.
본관은 덕수(德水), 자는 여해(汝諧), 시호는 충무(忠武)였으며, 한성 출신이었다.
문반 가문 출신으로 1576년(선조 9년) 무과(武科)에 급제하여 그 관직이 동구비보 권관, 훈련원 봉사, 발포진 수군만호, 조산보 만호, 전라좌도 수군절도사를 거쳐 정헌대부 삼도수군통제사에 이르렀다.
"""

print(nlp(question="이순신이 태어난 날짜는?", context=context))
print(nlp(question="이순신의 본관은?", context=context))
print(nlp(question="이순신의 시호는?", context=context))
print(nlp(question="이순신의 고향은?", context=context))
print(nlp(question="이순신의 마지막 직책은?", context=context))

{'score': 0.02416418306529522, 'start': 95, 'end': 98, 'answer': '충무('}
{'score': 0.6037680506706238, 'start': 72, 'end': 75, 'answer': '덕수('}
{'score': 0.19692738354206085, 'start': 95, 'end': 98, 'answer': '충무('}
{'score': 0.07258158177137375, 'start': 83, 'end': 86, 'answer': '여해('}
{'score': 0.0592343844473362, 'start': 214, 'end': 222, 'answer': '삼도수군통제사에'}
